In [1]:
import pandas as pd 
from sqlalchemy import create_engine
from datetime import timedelta, datetime, tzinfo
from zipfile import ZipFile
import collections

In [2]:
#data frame for legal status and pop
legal_df = pd.read_csv("legaldata.csv")

In [3]:
legal_df.head(3)

,State,Pop,legalWeedStatus,medicinalWeedStatus,decriminalizedWeedStatus,state
0,Alabama,4908621,Fully Illegal,False,False,Alabama
1,Alaska,734002,Fully Legal,True,True,Alaska
2,Arizona,7378494,Mixed,True,False,Arizona


In [4]:
#data frame for legal status and pop
MVR_df = pd.read_csv("mvdeaths.csv")

In [5]:
MVR_df.head(4)

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles"
0,2017,U.S. Total,"40,231","37,133",12.4,1.4,1.2
1,2017,Alabama,"1,034",948,21.2,1.9,1.3
2,2017,Alaska,101,79,13.7,1.0,1.4
3,2017,Arizona,"1,017","1,000",14.5,1.7,1.5


In [6]:
#data frame for legal status and pop
overdose_df = pd.read_csv("overdose.csv")

In [7]:
overdose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29568 entries, 0 to 29567
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State                          29568 non-null  object 
 1   Year                           29568 non-null  int64  
 2   Month                          29568 non-null  object 
 3   Period                         29568 non-null  object 
 4   Indicator                      29568 non-null  object 
 5   Data Value                     24236 non-null  float64
 6   Percent Complete               29568 non-null  object 
 7   Percent Pending Investigation  29568 non-null  float64
 8   State Name                     29568 non-null  object 
 9   Footnote                       29568 non-null  object 
 10  Footnote Symbol                29568 non-null  object 
 11  Predicted Value                18465 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usag

In [8]:
###filter clean to only columns I need
#states, indicator, year
od_df = overdose_df.drop(columns=["Period","Percent Complete","Percent Pending Investigation","Footnote","Footnote Symbol","Footnote Symbol","Predicted Value"])
od_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29568 entries, 0 to 29567
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       29568 non-null  object 
 1   Year        29568 non-null  int64  
 2   Month       29568 non-null  object 
 3   Indicator   29568 non-null  object 
 4   Data Value  24236 non-null  float64
 5   State Name  29568 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ MB


In [9]:
###Total deaths

totaldeaths = od_df[od_df['Indicator']=='Number of Deaths']
totaldeaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2968 entries, 4 to 29562
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       2968 non-null   object 
 1   Year        2968 non-null   int64  
 2   Month       2968 non-null   object 
 3   Indicator   2968 non-null   object 
 4   Data Value  2968 non-null   float64
 5   State Name  2968 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 162.3+ KB


In [27]:
totaldeaths['Data Value'] = totaldeaths['Data Value'].astype(str).astype(int)
totaldeaths.info()


ValueError: invalid literal for int() with base 10: '4,133'

In [24]:
yr_deaths = totaldeaths.groupby(['State','Year','Indicator','State Name'])['Data Value'].sum().reset_index()
yr_deaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2968 entries, 4 to 29562
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   State       2968 non-null   object
 1   Year        2968 non-null   int64 
 2   Month       2968 non-null   object
 3   Indicator   2968 non-null   object
 4   Data Value  2968 non-null   object
 5   State Name  2968 non-null   object
dtypes: int64(1), object(5)
memory usage: 162.3+ KB


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [23]:
od_df['Data Value'] = pd.to_numeric(od_df['Data Value'])




ValueError: Unable to parse string "4,133" at position 4

In [9]:
totaldeaths.head(5)

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
4,AK,2015,April,12 month-ending,Number of Deaths,"4,133",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
16,AK,2015,August,12 month-ending,Number of Deaths,"4,222",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
30,AK,2015,December,12 month-ending,Number of Deaths,"4,193",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
45,AK,2015,February,12 month-ending,Number of Deaths,"4,084",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
49,AK,2015,January,12 month-ending,Number of Deaths,"4,034",100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN


In [ ]:
##clean up data for each

In [3]:
#rds_connection_string = f"{dbuser}<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')
password = "Hooch#12"
dbuser = "postgres"
rds_connection_string = f"{dbuser}:{password}@localhost:5432/dfauibpirrt36u"
engine = create_engine(f'postgresql://{rds_connection_string}')

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
###postgresSQL
##push to server

In [ ]:
engine.table_names()

In [ ]:
#pushing Cluster,Bar,Line DATAFRAME to SQL
years_trend.to_sql(name='guns_year', con=engine, if_exists='replace', index=False)
deaths_2018_unindexed.to_sql(name='killings_injuries_2018', con=engine, if_exists='replace', index=False)

In [ ]:

#Creating CSV's of Cluster,Bar,Line DATAFRAME
years_trend.to_csv('year_trend_gun.csv')
deaths_2018.to_csv('deaths_2018.csv')
gun14_monthly.to_csv("gun2014onwrd.csv")